<a href="https://colab.research.google.com/github/YogeshGadade/Analytics/blob/master/FIFA_World_Cup_2022_Twitter_Data_Sentiment_Analysis_and_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



### About FIFA 2022: 
- World Cup 2022 Dates: Nov 20, 2022 – Dec 18, 2022
- The FIFA World Cup, a global football sporting event that takes place every four years, was in Qatar.


## Problems Addressed:
### 1. Sentiment Analysis
#### Problem:
- The decision to hold the World Cup in Qatar has sparked several controversies, including allegations of corruption and human rights violations.

#### Solution: 
- So, what do football lovers think about the FIFA World Cup 2022? To find out that, I’ll perform a Twitter sentiment analysis using the hashtag #WorldCup2022 on Twitter in this blog post.

### 2. Predictions
Answering Following Questions:
1. Champion Team
2. Best Player(s)
3. Top Scorer(s) 

Answering Following Questions: 
1. Which team has the most wins?
2. What is the winning percentage comparing when the highest-ranked team plays against the lowest-ranked team?
3. What is the winning percentage comparing when the highest attack rank plays against the lowest attack rank?
4. Do teams with stronger offensive scores have more goals?
5. Do teams with stronger goalkeepers receive fewer goals?
6. Team with the longest win streak?
7. Better team win percentage?
8. which team has the best goalkeeper, strongest defense, midfield, and offense score?
9. Teams with a high win percentage?
10. Winner prediction FIFA world cup 2022


## Performance of Finals Prediction
1. Actual Results Vs Predictions 
